In [1]:
import pandas as pd
import numpy as np

In [2]:
# data = pd.read_csv('../data/data_nc.nctools-predictions.txt', sep='\t')
# data['#Chrom'] = data['#Chrom'].apply(lambda x: str(x))
# data.shape

In [3]:
data = pd.read_csv('../data/data_nc.txt', sep='\t')
data['#Chrom'] = data['#Chrom'].apply(lambda x: str(x))
data.shape

//anaconda3/envs/py3_vp/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (0,4,5,6,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,134,140,141,142,143,144,146,147,148,149,152,153,159) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(82188, 160)

In [4]:
data.label.value_counts()

Benign        79577
Pathogenic     2611
Name: label, dtype: int64

In [5]:
data.NCBoost_data.value_counts()

False    81536
True       652
Name: NCBoost_data, dtype: int64

In [6]:
data_snvs = data[data.Ref.apply(lambda x: len(x) == 1) & data.Alt.apply(lambda x: len(x) == 1)]
data_snvs.label.value_counts()

Benign        69055
Pathogenic     2369
Name: label, dtype: int64

In [7]:
print(data.shape)
print(data_snvs.shape)
print(data.shape[0] - data_snvs.shape[0], 'non SNVs.')

(82188, 160)
(71424, 160)
10764 non SNVs.


Add the predictions of the nctools to the train set. For ReMM and NCBoost, the predictions are position based for SNV's. Therefore, they can only be merged with SNV variants from the training set. 

<h1>Add ReMM predictions to train set</h1>

As the ReMM scores are comupted only for SNVs and don't contain the alleles in the file, first merge the ReMM df with a subset of the train set only containing SNV's. Then merge with the total train set. 

In [8]:
remm = pd.read_csv('../data/nctools_predictions/ReMM_filtered.txt', sep='\s')
print(data.shape)

//anaconda3/envs/py3_vp/lib/python3.7/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


(82188, 160)


In [9]:
remm.columns = ['#Chrom', 'Pos', 'ReMM_score']
print(remm.shape)
remm = remm.drop_duplicates()
print(remm.shape)
remm.head()

(155268, 3)
(77636, 3)


,#Chrom,Pos,ReMM_score
0,1,874816,0.253
1,1,899926,0.490
2,1,905775,0.995
3,1,978847,0.936
4,1,980669,0.247


In [10]:
remm_data = data.merge(remm, on=['#Chrom', 'Pos'], how='left')

In [11]:
print(remm_data.shape)
remm_data.head()

(82188, 161)


,#Chrom,Pos,Ref,Alt,NCBoost_closest_gene_name,NCBoost_annovar_genomic_region,NCBoost_OMIM_diseases,NCBoost_HGMD_DM_public,NCBoost_ClinVar,NCBoost_Smedley,...,max_AF,notinTest1,oldTestset,provean,revel,sample_weight,sift,source,to_be_deleted,ReMM_score
0,1,21835920,C,T,ALPL,UTR5,"Hypophosphatasia, adult |||| Odontohypophospha...",1.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.925
1,1,55505180,C,A,PCSK9,UTR5,"Hypercholesterolemia, familial, 3",1.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.611
2,1,113498814,C,T,SLC16A1,UTR5,Erythrocyte lactate transporter defect |||| Hy...,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.987
3,1,209975332,G,T,IRF6,UTR5,van der Woude syndrome |||| Popliteal pterygiu...,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.931
4,1,209975361,T,A,IRF6,UTR5,van der Woude syndrome |||| Popliteal pterygiu...,1.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.692


ReMM only provides scores for SNVs. Therefore remove the ReMM scores from MNVs in the data. 

In [12]:
print(data.shape)
print(remm_data.shape)
print('Number of SNVs', len(np.where((remm_data.Ref.apply(lambda x: len(x) == 1)) & (remm_data.Alt.apply(lambda x: len(x) == 1)))[0]))
print('Number of MNVs', len(np.where((remm_data.Ref.apply(lambda x: len(x) != 1)) | (remm_data.Alt.apply(lambda x: len(x) != 1)))[0]))

(82188, 160)
(82188, 161)
Number of SNVs 71424
Number of MNVs 10764


In [13]:
remm_data.loc[np.where((remm_data.Ref.apply(lambda x: len(x) != 1)) | (remm_data.Alt.apply(lambda x: len(x) != 1)))[0], 'ReMM_score'] = np.nan

In [14]:
remm_data.ReMM_score.isna().sum()

10764

In [15]:
data = remm_data

<h1>Add NCBoost predictions to train set</h1>

In [16]:
# data_snvs = data[data.Ref.apply(lambda x: len(x) == 1) & data.Alt.apply(lambda x: len(x) == 1)]
# data_snvs['#Chrom'] = data_snvs['#Chrom'].apply(lambda x: str(x)) #convert all chromosmes to string (some are int)
# data_snvs = data_snvs[['#Chrom', 'Pos', 'Ref', 'Alt', 'GeneName']] 
# print(data_snvs.shape)
# data_snvs.head()

In [17]:
ncboost = pd.read_csv('../data/nctools_predictions/data_nc.ncboost.annotated.tsv', sep='\t')
print(data.shape, ncboost.shape)
print(ncboost.columns.tolist())
ncboost = ncboost[['chr', 'pos', 'ref', 'alt', 'closest_gene_name', 'NCBoost']]
ncboost.columns = ['#Chrom', 'Pos', 'Ref', 'Alt', 'NCBoost_predictions_closest_gene_name', 'NCBoost_Score']
ncboost['#Chrom'] = ncboost['#Chrom'].apply(lambda x: str(x)) #convert all chromosmes to string (some are int)
ncboost.head()

(82188, 161) (42402, 54)
['chr', 'pos', 'ref', 'alt', 'annovar_annotation', 'closest_gene_name', 'gene_type', 'pLI', 'familyMemberCount', 'ncRVIS', 'ncGERP', 'RVIS_percentile', 'slr_dnds', 'GDI', 'gene_age', 'UTR3', 'UTR5', 'downstream', 'intergenic', 'intronic', 'upstream', 'GC', 'CpG', 'priPhCons', 'mamPhCons', 'verPhCons', 'priPhyloP', 'mamPhyloP', 'verPhyloP', 'GerpN', 'GerpS', 'bStatistic', 'TajimasD_YRI_pvalue', 'TajimasD_CEU_pvalue', 'TajimasD_CHB_pvalue', 'FuLisD_YRI_pvalue', 'FuLisD_CEU_pvalue', 'FuLisD_CHB_pvalue', 'FuLisF_YRI_pvalue', 'FuLisF_CEU_pvalue', 'FuLisF_CHB_pvalue', 'meanDaf1000G', 'meanHet1000G', 'meanMAF1000G', 'meanMAFGnomAD', 'meanMAF_AFRGnomAD', 'meanMAF_AMRGnomAD', 'meanMAF_ASJGnomAD', 'meanMAF_EASGnomAD', 'meanMAF_FINGnomAD', 'meanMAF_NFEGnomAD', 'meanMAF_OTHGnomAD', 'CDTS', 'NCBoost']


//anaconda3/envs/py3_vp/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,#Chrom,Pos,Ref,Alt,NCBoost_predictions_closest_gene_name,NCBoost_Score
0,1,980669,A,G,AGRN,0.208069
1,1,980948,C,T,AGRN,0.147836
2,1,981087,A,G,AGRN,0.183710
3,1,981328,C,G,AGRN,0.311922
4,1,982356,G,A,AGRN,0.145453


In [18]:
print(ncboost.shape)
ncboost = ncboost.drop_duplicates()
print(ncboost.shape)

(42402, 6)
(42370, 6)


In [19]:

print('Merging NCBoost data on Chrom, Pos and GeneName')
print(data.shape)
ncboost_data = data.merge(ncboost, right_on=['#Chrom', 'Pos', 'NCBoost_predictions_closest_gene_name'], left_on=['#Chrom', 'Pos', 'GeneName'], how='left')
print(ncboost_data.shape)
print('Percentage of variants without NCBoost score:', round(ncboost_data.NCBoost_Score.isna().sum()/ncboost_data.shape[0]*100,2))

print('\nMerging NCBoost data on Chrom, Pos, GeneName, Ref and Alt')
print(data.shape)
ncboost_data = data.merge(ncboost, right_on=['#Chrom', 'Pos', 'Ref', 'Alt', 'NCBoost_predictions_closest_gene_name'], left_on=['#Chrom', 'Pos', 'Ref', 'Alt', 'GeneName'], how='left')
print(ncboost_data.shape)
print('Percentage of variants without NCBoost score:', round(ncboost_data.NCBoost_Score.isna().sum()/ncboost_data.shape[0]*100,2))

print('\nMerging NCBoost data on Chrom, Pos')
print(data.shape)
ncboost_data = data.merge(ncboost, right_on=['#Chrom', 'Pos'], left_on=['#Chrom', 'Pos'], how='left')
print(ncboost_data.shape)

print('Percentage of variants without NCBoost score:', round(ncboost_data.NCBoost_Score.isna().sum()/ncboost_data.shape[0]*100,2))


Merging NCBoost data on Chrom, Pos and GeneName
(82188, 161)
(83215, 165)
Percentage of variants without NCBoost score: 65.05

Merging NCBoost data on Chrom, Pos, GeneName, Ref and Alt
(82188, 161)
(82188, 163)
Percentage of variants without NCBoost score: 66.22

Merging NCBoost data on Chrom, Pos
(82188, 161)
(84564, 165)
Percentage of variants without NCBoost score: 51.93


There are multiple ways to merge them. The authors of NCBoost said that the scores are position specific, not allele specific. However, NCBoost does give multiple scores for the same variant related to different genes (manually scanning the file they don't seem to differ much). Therefore, we can merge them on chromosome, position, gene name and alleles, or only on chromosome and position. The first one will be more correct but the second one yields more scores for variants. 

In [20]:
print('Merging NCBoost data on Chrom, Pos, GeneName, Ref and Alt')
print(data.shape)
ncboost_data = data.merge(ncboost, right_on=['#Chrom', 'Pos', 'Ref', 'Alt', 'NCBoost_predictions_closest_gene_name'], left_on=['#Chrom', 'Pos', 'Ref', 'Alt', 'GeneName'], how='left')
print(ncboost_data.shape)


Merging NCBoost data on Chrom, Pos, GeneName, Ref and Alt
(82188, 161)
(82188, 163)


In [21]:
data = ncboost_data
data.shape

(82188, 163)

In [22]:
print('Merging NCBoost data on Chrom and Pos')
print(ncboost.shape)
ncboost = ncboost.rename({'NCBoost_Score': 'NCBoost_Score-merged-chrom-pos'}, axis='columns')
ncboost_dropped_duplicates = ncboost.drop_duplicates(subset=['#Chrom', 'Pos'])
print(ncboost_dropped_duplicates.shape)
ncboost_data = data.merge(ncboost_dropped_duplicates[['#Chrom', 'Pos', 'NCBoost_Score-merged-chrom-pos']], on=['#Chrom', 'Pos'], how='left')
print(ncboost_data.shape)

Merging NCBoost data on Chrom and Pos
(42370, 6)
(41271, 6)
(82188, 164)


In [23]:
data = ncboost_data

In [24]:
data.loc[np.where((data.Ref.apply(lambda x: len(x) != 1)) | (data.Alt.apply(lambda x: len(x) != 1)))[0], 'NCBoost_Score'] = np.nan
data.loc[np.where((data.Ref.apply(lambda x: len(x) != 1)) | (data.Alt.apply(lambda x: len(x) != 1)))[0], 'NCBoost_Score-merged-chrom-pos'] = np.nan

<h3>Add NCBoost precomputed scores</h3>

In [25]:
ncboost_precomputed = pd.read_csv('../data/nctools_predictions/NCBoost_precomputed_predictions.txt', sep='\t')
ncboost_precomputed.columns = ['#Chrom', 'Pos', 'region', 'NCBoost_precomputed_closest_gene_name', 'NCBoost_precomputed_score', 'NCBoost_precomputed_chr_rank_perc']
print(ncboost_precomputed.shape)
ncboost_precomputed.head()

(36121, 6)


,#Chrom,Pos,region,NCBoost_precomputed_closest_gene_name,NCBoost_precomputed_score,NCBoost_precomputed_chr_rank_perc
0,1,899926,intronic,KLHL17,0.095736,0.758251
1,1,980669,intronic,AGRN,0.164826,0.866997
2,1,980948,intronic,AGRN,0.111841,0.792821
3,1,981087,intronic,AGRN,0.131175,0.825299
4,1,981328,intronic,AGRN,0.255828,0.930296


In [26]:
print('Merging NCBoost data on Chrom, Pos and GeneName')
ncboost_data = data.merge(ncboost_precomputed[['#Chrom', 'Pos', 'NCBoost_precomputed_score', 'NCBoost_precomputed_closest_gene_name']].drop_duplicates(['#Chrom', 'Pos', 'NCBoost_precomputed_closest_gene_name']), right_on=['#Chrom', 'Pos', 'NCBoost_precomputed_closest_gene_name'], left_on=['#Chrom', 'Pos', 'GeneName'], how='left')
print(ncboost_data.shape)
print('Percentage of variants without NCBoost score:', round(ncboost_data.NCBoost_precomputed_score.isna().sum()/ncboost_data.shape[0]*100,2))

print('\nMerging NCBoost data on Chrom and Pos')
print(data.shape)
ncboost_data = data.merge(ncboost_precomputed[['#Chrom', 'Pos', 'NCBoost_precomputed_score']].drop_duplicates(['#Chrom', 'Pos']), right_on=['#Chrom', 'Pos'], left_on=['#Chrom', 'Pos'], how='left')
print(ncboost_data.shape)
print('Percentage of variants without NCBoost score:', round(ncboost_data.NCBoost_precomputed_score.isna().sum()/ncboost_data.shape[0]*100,2))


Merging NCBoost data on Chrom, Pos and GeneName
(82188, 166)
Percentage of variants without NCBoost score: 65.48

Merging NCBoost data on Chrom and Pos
(82188, 164)
(82188, 165)
Percentage of variants without NCBoost score: 54.81


In [27]:
print('Merging NCBoost data on Chrom, Pos and GeneName')
ncboost_data = data.merge(ncboost_precomputed[['#Chrom', 'Pos', 'NCBoost_precomputed_score', 'NCBoost_precomputed_chr_rank_perc', 'NCBoost_precomputed_closest_gene_name']].drop_duplicates(['#Chrom', 'Pos', 'NCBoost_precomputed_closest_gene_name']), right_on=['#Chrom', 'Pos', 'NCBoost_precomputed_closest_gene_name'], left_on=['#Chrom', 'Pos', 'GeneName'], how='left')
print(ncboost_data.shape)

Merging NCBoost data on Chrom, Pos and GeneName
(82188, 167)


In [28]:
data = ncboost_data

In [29]:
print('Merging NCBoost data on Chrom and Pos')
ncboost_precomputed = ncboost_precomputed.rename({'NCBoost_precomputed_score': 'NCBoost_precomputed_score-merged-chrom-pos', 'NCBoost_precomputed_chr_rank_perc': 'NCBoost_precomputed_chr_rank_perc-merged-chrom-pos'}, axis=1)
ncboost_data = data.merge(ncboost_precomputed[['#Chrom', 'Pos', 'NCBoost_precomputed_score-merged-chrom-pos', 'NCBoost_precomputed_chr_rank_perc-merged-chrom-pos']].drop_duplicates(['#Chrom', 'Pos']), right_on=['#Chrom', 'Pos'], left_on=['#Chrom', 'Pos'], how='left')
ncboost_data.shape

Merging NCBoost data on Chrom and Pos


(82188, 169)

In [30]:
data = ncboost_data

In [31]:
data.loc[np.where((data.Ref.apply(lambda x: len(x) != 1)) | (data.Alt.apply(lambda x: len(x) != 1)))[0], 'NCBoost_precomputed_score'] = np.nan
data.loc[np.where((data.Ref.apply(lambda x: len(x) != 1)) | (data.Alt.apply(lambda x: len(x) != 1)))[0], 'NCBoost_precomputed_score-merged-chrom-pos'] = np.nan
data.loc[np.where((data.Ref.apply(lambda x: len(x) != 1)) | (data.Alt.apply(lambda x: len(x) != 1)))[0], 'NCBoost_precomputed_chr_rank_perc'] = np.nan
data.loc[np.where((data.Ref.apply(lambda x: len(x) != 1)) | (data.Alt.apply(lambda x: len(x) != 1)))[0], 'NCBoost_precomputed_chr_rank_perc-merged-chrom-pos'] = np.nan

<h3>NCBoost scores summary</h3>

In [32]:
for x in data.columns:
    if 'NCBoost_Score' in x or 'NCBoost_precomputed' in x:
       print(data.shape[0] - data[x].isna().sum(), 'variants for', x)

27760 variants for NCBoost_Score
38391 variants for NCBoost_Score-merged-chrom-pos
23795 variants for NCBoost_precomputed_score
23795 variants for NCBoost_precomputed_chr_rank_perc
28371 variants for NCBoost_precomputed_closest_gene_name
31542 variants for NCBoost_precomputed_score-merged-chrom-pos
31542 variants for NCBoost_precomputed_chr_rank_perc-merged-chrom-pos


<h1>Add DeepSEA predictions to train set</h1>

Here, the functional significance DeepSEA scores are added to the train set.

TODO: also use the DeepSEA classifiers for the variants in the train set.

See: http://deepsea.princeton.edu/help/. "We provide three classifiers trained on HGMD mutations, eQTLs and GWAS lead SNPs respectively. The first is likely appropriate for de novo mutation while the later two are likely appropriate for common variants. We also provide an unsupervised score named functional signficance score - it is not biased by any variant training set but may not have as strong performance as classifiers."

In [33]:
deepsea = pd.read_csv('../data/nctools_predictions/deepsea_jobs/infile.vcf.out.funsig', index_col=0)
deepsea = deepsea.drop(['name'], axis=1)
deepsea.columns = ['#Chrom', 'Pos', 'Ref', 'Alt', 'DeepSEA_Functional_significance_score']
deepsea['#Chrom'] = deepsea['#Chrom'].apply(lambda x: x[3:])
deepsea.head()

,#Chrom,Pos,Ref,Alt,DeepSEA_Functional_significance_score
72601,8,145008443,A,T,0.000010
40518,19,11200088,C,T,0.000013
40514,19,11200085,C,T,0.000015
73328,9,35657869,C,T,0.000016
67927,6,152702540,C,T,0.000017


In [34]:
merged = data.merge(deepsea, on=['#Chrom', 'Pos', 'Ref', 'Alt'], how='left')

In [35]:
merged.shape

(82188, 170)

In [36]:
data = merged

<h1>Add Eigen predictions to train set</h1>

In [37]:
eigen = pd.read_csv('../data/nctools_predictions/eigen_filtered.txt', sep='\t')
eigen

//anaconda3/envs/py3_vp/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (6,7,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,chr,position,ref,alt,GERP_NR,GERP_RS,PhyloPri,PhyloPla,PhyloVer,PhastPri,...,PolIISig,PolIIPval,ctcfSig,ctcfPval,cmycSig,cmycPval,Eigen-raw,Eigen-phred,Eigen-PC-raw,Eigen-PC-phred
0,1,874714,C,A,3.83,1.7100,0.591,0.938,0.326,0.008,...,0.1700,8.48,3.8276,16.00,0.1533,6.42,1.895022,27.5634,5.274241,27.0578
1,1,899926,G,A,2.74,-5.4700,0.109,-0.856,1.702,0.387,...,0.0770,4.15,0.0815,1.12,0.1704,2.66,0.562761,12.1877,2.419997,18.6624
2,1,899926,G,C,2.74,-5.4700,0.109,-0.856,1.702,0.387,...,0.0770,4.15,0.0815,1.12,0.1704,2.66,0.562761,12.1877,2.419997,18.6624
3,1,899926,G,T,2.74,-5.4700,0.109,-0.856,1.702,0.387,...,0.0770,4.15,0.0815,1.12,0.1704,2.66,0.562761,12.1877,2.419997,18.6624
4,1,978847,G,A,4.13,4.1300,0.655,2.136,2.035,0.087,...,0.0535,2.86,0.4269,3.62,0.1176,2.02,1.781010,25.2996,3.432827,21.3394
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
631218,22,51137249,T,C,4.11,0.0819,-0.282,-0.359,-2.621,0.006,...,0.0773,2.83,2.1540,12.80,0.1149,3.63,0.539487,11.9861,2.551899,19.0133
631219,22,51137249,T,G,4.11,0.0819,-0.282,-0.359,-2.621,0.006,...,0.0773,2.83,2.1540,12.80,0.1149,3.63,0.539487,11.9861,2.551899,19.0133
631220,22,51182485,A,C,4.48,0.9820,-0.404,0.143,-0.032,0.528,...,0.0282,1.40,0.0541,0.98,0.0530,2.82,0.575655,12.2964,0.804696,14.0648
631221,22,51182485,A,G,4.48,0.9820,-0.404,0.143,-0.032,0.528,...,0.0282,1.40,0.0541,0.98,0.0530,2.82,0.575655,12.2964,0.804696,14.0648


In [38]:
eigen = eigen[['chr', 'position', 'ref', 'alt', 'Eigen-raw', 'Eigen-phred', 'Eigen-PC-raw', 'Eigen-PC-phred']]

In [40]:
eigen.columns = ['#Chrom', 'Pos', 'Ref', 'Alt', 'Eigen-raw', 'Eigen-phred', 'Eigen PC-raw', 'Eigen PC-phred']

In [41]:
eigen['#Chrom'] = eigen['#Chrom'].apply(lambda x: str(x))

//anaconda3/envs/py3_vp/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [42]:
eigen.head()

,#Chrom,Pos,Ref,Alt,Eigen-raw,Eigen-phred,Eigen PC-raw,Eigen PC-phred
0,1,874714,C,A,1.895022,27.5634,5.274241,27.0578
1,1,899926,G,A,0.562761,12.1877,2.419997,18.6624
2,1,899926,G,C,0.562761,12.1877,2.419997,18.6624
3,1,899926,G,T,0.562761,12.1877,2.419997,18.6624
4,1,978847,G,A,1.781010,25.2996,3.432827,21.3394


In [43]:
print(eigen.shape)
eigen = eigen.drop_duplicates()
print(eigen.shape)

(631223, 8)
(211084, 8)


In [44]:
merged = data.merge(eigen, on=['#Chrom', 'Pos', 'Ref', 'Alt'], how='left')

In [45]:
merged.shape

(82188, 174)

In [46]:
data = merged

<h3>Merge Eigen scores only on chrom and pos</h3>

In [47]:
print(eigen.shape)
eigen = eigen[['#Chrom', 'Pos', 'Eigen-raw', 'Eigen PC-raw']]
eigen.columns = ['#Chrom', 'Pos', 'Eigen-raw_merged-chr-pos', 'Eigen PC-raw_merged-chr-pos']
eigen['#Chrom'] = eigen['#Chrom'].apply(lambda x: str(x)) #convert all chromosmes to string (some are int)
eigen.head()

(211084, 8)


,#Chrom,Pos,Eigen-raw_merged-chr-pos,Eigen PC-raw_merged-chr-pos
0,1,874714,1.895022,5.274241
1,1,899926,0.562761,2.419997
2,1,899926,0.562761,2.419997
3,1,899926,0.562761,2.419997
4,1,978847,1.781010,3.432827


In [48]:
eigen = eigen.drop_duplicates(subset=['#Chrom', 'Pos'])
eigen.shape

(70822, 4)

In [50]:
eigen_data = data.merge(eigen, on=['#Chrom', 'Pos'], how='left')

In [51]:
eigen_data.shape

(82188, 176)

In [52]:
data = eigen_data

In [53]:
data.loc[np.where((data.Ref.apply(lambda x: len(x) != 1)) | (data.Alt.apply(lambda x: len(x) != 1)))[0], 'Eigen-raw_merged-chr-pos'] = np.nan
data.loc[np.where((data.Ref.apply(lambda x: len(x) != 1)) | (data.Alt.apply(lambda x: len(x) != 1)))[0], 'Eigen-PC-raw_merged-chr-pos'] = np.nan

<h3>Add LiNSIGHT scores</h3>

In [54]:
linsight = pd.read_csv('../data/nctools_predictions/linsight_predictions.txt', sep='\t')
linsight = linsight[['#Chrom', 'Pos', 'LINSIGHT_score']]
print(linsight.shape)
linsight = linsight.drop_duplicates()
print(linsight.shape)
linsight.head()

(71421, 3)
(68618, 3)


,#Chrom,Pos,LINSIGHT_score
0,1,21835920,0.113951
1,1,55505180,0.214607
2,1,113498814,0.055969
3,1,209975332,0.160774
4,1,209975361,0.081169


In [55]:
#The LiNSIGHT scores are position based, so only merge on chromosome and position. 

In [56]:
linsight_data = data.merge(linsight, on=['#Chrom', 'Pos'], how='left')

In [57]:
linsight_data.shape

(82188, 178)

In [58]:
data = linsight_data

In [59]:
data.loc[np.where((data.Ref.apply(lambda x: len(x) != 1)) | (data.Alt.apply(lambda x: len(x) != 1)))[0], 'LINSIGHT_score'] = np.nan


<h1>Save train set with added columns of NCtools</h1>

In [60]:
data.to_csv('../data/data_nc.nctools-predictions.txt', sep='\t', index=False)